In [1]:
import sys
import math

import numpy as np

In [29]:
class Edge(object):
    def __init__(self, point1, point2):
        """ reprents a line between two points """
        self.point1 = point1
        self.point2 = point2

    def intersects(self, edge):
        """ returns True if we intersect or overlap the edge """
        #print(f's={self}, e={edge}', file=sys.stderr)
        if self.point1 == edge.point1:
            return self.point1
        if self.point2 == edge.point2:
            return self.point2

        xdiff = (self.point1[0] - self.point2[0], edge.point1[0] - edge.point2[0])
        ydiff = (self.point1[1] - self.point2[1], edge.point1[1] - edge.point2[1])
        def det(a, b):
            return a[0]*b[1] - a[1]*b[0]

        div = det(xdiff, ydiff)
        if div == 0:
            print(f'edge: {edge} does not intersect with {self}', file=sys.stderr)
            return False
        d = (det(self.point1, self.point2), det(edge.point1, edge.point2))
        # intersection point
        p = (det(d, xdiff)/div, det(d, ydiff)/div)

        # make sure we are within limits
        p1x = [self.point1[0], self.point2[0]]
        p2x = [edge.point1[0], edge.point2[0]]
        p1y = [self.point1[1], self.point2[1]]
        p2y = [edge.point1[1], edge.point2[1]]
        
        if p[0] < max([min(p1x), min(p2x)]) or p[0] > min([max(p1x), max(p2x)]):
            return False
        if p[1] < max([min(p1y), min(p2y)]) or p[1] > min([max(p1y), max(p2y)]):
            return False
        return p

    def norm(self):
        """ returns L2 norm """
        return math.sqrt((self.point2[1]-self.point1[1])**2 + (self.point2[0]-self.point1[0])**2)
    
    def __repr__(self):
        return f'({self.point1},{self.point2})'

    
    
class Polygon(object):
    def __init__(self, vertices):
        """ input is a list of non-ordered vertices anti-clockwise starting negative y-axis of area (SE) """
        #print(f'input: {vertices}', file=sys.stderr)
        def sort_vertices(vertices):
            x_center = sum([x[0] for x in vertices])/len(vertices)
            y_center = sum([x[1] for x in vertices])/len(vertices)
            angles = [math.atan2(v[1]-y_center, v[0]-x_center) for v in vertices]
            indices = np.argsort(angles)
            #print(f'angles: {angles}', file=sys.stderr)
            v = np.array(vertices)
            return tuple(v[indices].tolist())

        self.vertices = sort_vertices(vertices)

    def min_y(self):
        return min([x[1] for x in self.vertices])

    def contains(self, point):
        """ point is a tuple """
        # (x-x1)*(y2-y1)-(y-y1)*(x2-x1)
        xprod = [
            (point[0] - edge.point1[0])*(edge.point2[1]-edge.point1[1]) - (point[1] - edge.point1[1])*(edge.point2[0]-edge.point1[0])
            for edge in self.edges()
        ]
        #print(f'check if {point} is in {self}:xprod={xprod}', file=sys.stderr)
        return all(map(lambda x:x<=0, xprod))

    def center(self):
        """ center of polygon """
        n = len(self.vertices)
        return (sum([x[0] for x in self.vertices])/n, sum([x[1] for x in self.vertices])/n)
        
    def area(self):
        """ shoelace algorithm """
#         n = len(self.vertices)
#         return 0.5*abs(sum([
#             (self.vertices[ndx][0]*self.vertices[(ndx+1)%n][1])-(self.vertices[ndx][1]*self.vertices[(ndx+1)%n][0])
#             for ndx in range(n)
#         ]))
        # return (sum([
        #     (self.vertices[ndx][0] + self.vertices[(ndx+1)%n][0])*(self.vertices[ndx][1] + self.vertices[(ndx+1)%n][1])
        #     for ndx in range(n)
        # ]))
        center = self.center()
        triangles = [(center, x.point1, x.point2) for x in self.edges()]
        print(f'triangles: {triangles}', file=sys.stderr)
        def triangle_area(x):
            (a,b,c) = x
            return abs(((a[0]-b[0])*(c[1]-a[1]) - (a[0]-c[0])*(b[1]-a[1])))/2
        print(f'areas: {list(map(triangle_area, triangles))}', file=sys.stderr)

        return sum(map(triangle_area, triangles))

    def area2(self):
        """ shoelace algorithm """
        n = len(self.vertices)
        return 0.5*abs(sum([
            (self.vertices[ndx][0]*self.vertices[(ndx+1)%n][1])-(self.vertices[ndx][1]*self.vertices[(ndx+1)%n][0])
            for ndx in range(n)
        ]))
    
    
    def perimeter(self):
        """ length of all the edges """
        return sum(map(Edge.norm, self.edges()))
    
    def edges(self):
        """ returns ordered pairs of vertices """
        return [
            Edge(self.vertices[ndx], self.vertices[(ndx+1)%len(self.vertices)])
            for ndx in range(len(self.vertices))
        ]

    def __repr__(self):
        return f'Poly({self.vertices})'



In [23]:
2*math.sqrt(math.pi*10)

11.209982432795858

In [24]:
poly = Polygon(([-0.36363636363636365, 1.4545454545454546], [1.0, 1.0], [1.0, 0.3333333333333333], [2.0, 0.6666666666666666], [3.0, 0.3333333333333333], [3.0, 1.0], [4.8, 1.6], [4.333333333333333, 2.6666666666666665], [3.0, 3.0], [3.0, 4.0], [1.6666666666666667, 3.3333333333333335], [1.0, 3.5], [1.0, 3.0], [-0.5555555555555556, 2.2222222222222223]))
print(poly.area())
print(poly.area2())

10.635353535353534
10.635353535353534


triangles: [((1.9914862914862914, 2.007864357864358), [-0.36363636363636365, 1.4545454545454546], [1.0, 1.0]), ((1.9914862914862914, 2.007864357864358), [1.0, 1.0], [1.0, 0.3333333333333333]), ((1.9914862914862914, 2.007864357864358), [1.0, 0.3333333333333333], [2.0, 0.6666666666666666]), ((1.9914862914862914, 2.007864357864358), [2.0, 0.6666666666666666], [3.0, 0.3333333333333333]), ((1.9914862914862914, 2.007864357864358), [3.0, 0.3333333333333333], [3.0, 1.0]), ((1.9914862914862914, 2.007864357864358), [3.0, 1.0], [4.8, 1.6]), ((1.9914862914862914, 2.007864357864358), [4.8, 1.6], [4.333333333333333, 2.6666666666666665]), ((1.9914862914862914, 2.007864357864358), [4.333333333333333, 2.6666666666666665], [3.0, 3.0]), ((1.9914862914862914, 2.007864357864358), [3.0, 3.0], [3.0, 4.0]), ((1.9914862914862914, 2.007864357864358), [3.0, 4.0], [1.6666666666666667, 3.3333333333333335]), ((1.9914862914862914, 2.007864357864358), [1.6666666666666667, 3.3333333333333335], [1.0, 3.5]), ((1.9914862

In [28]:
t = Polygon([(0,0), (0,1), (1,0), (1,1)])
print(t)
print(t.area())
print(t.perimeter())

Poly(([0, 0], [1, 0], [1, 1], [0, 1]))
1.0
4.0


triangles: [((0.5, 0.5), [0, 0], [1, 0]), ((0.5, 0.5), [1, 0], [1, 1]), ((0.5, 0.5), [1, 1], [0, 1]), ((0.5, 0.5), [0, 1], [0, 0])]
areas: [0.25, 0.25, 0.25, 0.25]


In [10]:
#t.intersect(Polygon([(0,0),(0,1),(1,1)]))
p2 = Polygon([(0,0), (0,1), (1,1)])

intersections = [
    e1.intersects(e2) for e1 in t.edges() for e2 in p2.edges()
]
print(intersections)

i2 = []
for p in [p for p in t.vertices if p2.contains(p)] + [p for p in p2.vertices if t.contains(p)] + intersections:
    if p and tuple(p) not in i2:
        i2.append(tuple(p))
        
print(i2)

overlap = Polygon(i2)
print(overlap)
print(overlap.area())


[[0, 0], False, (-0.0, -0.0), [1, 1], (1.0, 1.0), False, (1.0, 1.0), [1, 1], (0.0, 1.0), (0.0, 0.0), (-0.0, 1.0), [0, 1]]
[(0, 0), (1, 1), (0, 1)]
Poly(([0, 0], [1, 1], [0, 1]))
0.5


edge: ([1, 1],[0, 1]) does not intersect with ([0, 0],[1, 0])
edge: ([0, 1],[0, 0]) does not intersect with ([1, 0],[1, 1])


In [20]:
Polygon([(0,0), (4,0), (6,4), (6,6), (4,6), (0,4)]).area()

triangles: [((3.3333333333333335, 3.3333333333333335), [0, 0], [4, 0]), ((3.3333333333333335, 3.3333333333333335), [4, 0], [6, 4]), ((3.3333333333333335, 3.3333333333333335), [6, 4], [6, 6]), ((3.3333333333333335, 3.3333333333333335), [6, 6], [4, 6]), ((3.3333333333333335, 3.3333333333333335), [4, 6], [0, 4]), ((3.3333333333333335, 3.3333333333333335), [0, 4], [0, 0])]
areas: [6.666666666666667, 4.666666666666667, 2.666666666666667, 2.666666666666667, 4.666666666666667, 6.666666666666667]


28.000000000000004